In [1]:
# Packages for Web-Scraping
from requests import get
from bs4 import BeautifulSoup
from time import time
from time import sleep
from random import randint
from IPython.display import clear_output
from warnings import warn

# Packages for Saving File after Scraping
import numpy as np
import pandas as pd

In [2]:
# url 1st page info, response object, and sample
url = 'https://www.politifact.com/factchecks/list/?page=1'
response = get(url)
print(response.text[:500])


<!DOCTYPE html>
<html lang="en-US" dir="ltr">
<head>
<meta charset="utf-8">
<meta http-equiv="x-ua-compatible" content="ie=edge">
<meta name="viewport" content="width=device-width, initial-scale=1">
<title> | PolitiFact </title>
<meta name="description" content="PolitiFact is a fact-checking website that rates the accuracy of claims by elected officials and others on its Truth-O-Meter." />
<meta name="twitter:username" content="PolitiFact" />
<meta name="twitter:card" content="summary" />
<meta


In [3]:
# 1st page BeautifulSoup object, confirm type
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [4]:
# finding each row of data we want to scrape, confirm type, check length
statement_containers = html_soup.find_all('div', class_ = 'm-statement__content')
print(type(statement_containers))
print(len(statement_containers))

<class 'bs4.element.ResultSet'>
30


In [5]:
s = statement_containers[1]
s

<div class="m-statement__content">
<div class="m-statement__body">
<div class="m-statement__quote-wrap">
<div class="m-statement__quote">
<a href="/factchecks/2022/feb/14/tweets/no-onlyfans-isnt-recruiting-adult-film-stars-colle/">
A video shows that “OnlyFans has guys on campus encouraging your daughters to do porn for ‘financial freedom.’”
</a>
</div>
</div>
<div class="m-statement__meter">
<div class="c-image" style="padding-top: 89.49771689497716%;">
<img alt="false" class="c-image__thumb" height="196" src="https://static.politifact.com/img/meter-false-th.jpg" width="219"/>
<picture>
<img alt="false" class="c-image__original" height="196" src="https://static.politifact.com/img/meter-false.jpg" width="219"/>
</picture>
</div>
</div>
<footer class="m-statement__footer">
By Bill McCarthy • February 14, 2022
</footer>
</div>
</div>

In [6]:
s.find('div', class_ = 'm-statement__quote').get_text(strip=True)

'A video shows that “OnlyFans has guys on campus encouraging your daughters to do porn for ‘financial freedom.’”'

In [7]:
list_container = html_soup.find('ul', class_ = 'o-listicle__list')
list_items_container = html_soup.find_all('li', class_ = 'o-listicle__item')
print(type(list_items_container))
print(len(list_items_container))

<class 'bs4.element.ResultSet'>
30


In [8]:
statement_containers = html_soup.find_all('div', class_ = 'm-statement__quote')
print(type(statement_containers))
print(len(statement_containers))

<class 'bs4.element.ResultSet'>
30


Extractig the data for a single list item

In [9]:
first_item = list_items_container[0]
first_item

<li class="o-listicle__item">
<article class="m-statement m-statement--is-medium m-statement--false">
<div class="m-statement__author">
<div class="m-statement__avatar">
<div class="m-statement__image">
<div class="c-image" style="padding-top: 119.27710843373494%;">
<img class="c-image__thumb" height="99" src="https://static.politifact.com/CACHE/images/politifact/mugs/Twitter/5a09f91f97a11c38d58405becf218e8c.jpg" width="83"/>
<picture>
<img class="c-image__original" height="178" src="https://static.politifact.com/CACHE/images/politifact/mugs/Twitter/58840a801728fb16b65bf50c585680f2.jpg" width="166"/>
</picture>
</div>
</div>
</div>
<div class="m-statement__meta">
<a class="m-statement__name" href="/personalities/tweets/" title="Tweets">
Tweets
</a>
<div class="m-statement__desc">
stated on February 1, 2022 in a post:
</div>
</div>
</div>
<div class="m-statement__content">
<div class="m-statement__body">
<div class="m-statement__quote-wrap">
<div class="m-statement__quote">
<a href="/fa

In [10]:
#actual statement
first_item.find('div', class_ = 'm-statement__quote').a.text[1:-1]

'"Books banned in Texas include 1984, Maus, and The Handmaid\'s Tale, but not Mein Kampf."'

In [11]:
#statement source
first_item.find('div', class_ = 'm-statement__meta').a.text[1:-1]

'Tweets'

In [12]:
#statement source
first_item.find('div', class_ = 'm-statement__quote').a['href']

'/factchecks/2022/feb/14/tweets/those-titles-arent-banned-statewide-could-be-distr/'

In [13]:
#finding the statement label
first_item.find('div', class_ = 'm-statement__content').picture.img['alt']

'false'

Create the feature list for the first element 

In [14]:
# Lists to store the scraped data in
statement = []
source = []
link = []
veracity = []

# Extract data from individual container
for container in list_items_container:
# statement
    sta = container.find('div', class_ = 'm-statement__quote').a.text[1:-1]
    statement.append(sta)
# source
    sou = container.find('div', class_ = 'm-statement__meta').a.text[1:-1]
    source.append(sou)
# link
    lin = container.find('div', class_ = 'm-statement__quote').a['href']
    link.append(lin)
# veracity
    ver = container.find('div', class_ = 'm-statement__content').picture.img['alt']
    veracity.append(ver)

In [15]:
# 1st page df
test_df = pd.DataFrame(
    {'statement': statement,
     'source': source,
     'link': link,
     'veracity': veracity
})
print(test_df.info())
test_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   statement  30 non-null     object
 1   source     30 non-null     object
 2   link       30 non-null     object
 3   veracity   30 non-null     object
dtypes: object(4)
memory usage: 1.1+ KB
None


,statement,source,link,veracity
0,"""Books banned in Texas include 1984, Maus, and...",Tweets,/factchecks/2022/feb/14/tweets/those-titles-ar...,false
1,A video shows that “OnlyFans has guys on campu...,Tweets,/factchecks/2022/feb/14/tweets/no-onlyfans-isn...,false
2,“Nevada ranks 50th in election integrity ratin...,Jesse Haw,/factchecks/2022/feb/14/jesse-haw/fact-checkin...,barely-true
3,“The second booster has eight strains of HIV.”,Facebook posts,/factchecks/2022/feb/14/facebook-posts/covid-1...,pants-fire
4,Virginia’s economic growth “has stalled at les...,Glenn Youngkin,/factchecks/2022/feb/14/glenn-youngkin/has-vir...,mostly-true
5,COVID-19 vaccines caused a 40% increase in dea...,Bloggers,/factchecks/2022/feb/11/blog-posting/no-covid-...,false
6,COVID spelled backward is “divoc” which means ...,Facebook posts,/factchecks/2022/feb/11/facebook-posts/covid-s...,false
7,“New Jersey and Maryland produce more solar po...,Charlie Crist,/factchecks/2022/feb/11/charlie-crist/florida-...,barely-true
8,“We all know China created COVID.”,Dave McCormick,/factchecks/2022/feb/11/dave-mccormick/respons...,false
9,Arizona House bill to “decertify” 2020 results...,Bloggers,/factchecks/2022/feb/11/blog-posting/gop-lawma...,pants-fire


In [16]:
# Testing 5 pages
pages = [str(i) for i in range(1,5)]
start_time = time()
requests = 0
for _ in range(5):
# request goes here
    requests += 1
    sleep(randint(1,3))
    current_time = time()
    elapsed_time = current_time - start_time
    print('Request: {}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

Request: 5; Frequency: 0.49718085459976785 requests/s


In [17]:
# If above is problem-free, proceed to scrape all pages
# Scraping 834 pages takes about 4-5 hours

# Lists to store all the scraped data in
statement = []
source = []
link = []
veracity = []

# Preparing the monitoring of the loop
start_time = time()
requests = 0

# For every page in the interval
pages = [str(i) for i in range(1,834)]
for page in pages:

    # Make a get request
    response = get('https://www.politifact.com/factchecks/list/?page=' + page)

    # Pause the loop in random intervals so your IP address doesn't get banned
    sleep(randint(8,15))

    # Monitor the requests
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    # Parse the content of the request with BeautifulSoup
    page_html = BeautifulSoup(response.text, 'html.parser')

    # Select all the containers from a single page
    list_container = page_html.find('ul', class_ = 'o-listicle__list')
    list_items_container = html_soup.find_all('li', class_ = 'o-listicle__item')

    for container in list_items_container:
    # statement
        sta = container.find('div', class_ = 'm-statement__quote').a.text[1:-1]
        statement.append(sta)
    # source
        sou = container.find('div', class_ = 'm-statement__meta').a.text[1:-1]
        source.append(sou)
    # link
        lin = container.find('div', class_ = 'm-statement__quote').a['href']
        link.append(lin)
    # veracity
        ver = container.find('div', class_ = 'm-statement__content').picture.img['alt']
        veracity.append(ver)

Request:833; Frequency: 0.07692828062164547 requests/s


In [18]:
# make a dataframe
politifact_df = pd.DataFrame(
    {'statement': statement,
     'source': source,
     'link': link,
     'veracity': veracity
})
print(politifact_df.info())
politifact_df.tail(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24990 entries, 0 to 24989
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   statement  24990 non-null  object
 1   source     24990 non-null  object
 2   link       24990 non-null  object
 3   veracity   24990 non-null  object
dtypes: object(4)
memory usage: 781.1+ KB
None


,statement,source,link,veracity
24988,Comedian Heather McDonald collapsed on stage a...,Facebook posts,/factchecks/2022/feb/09/facebook-posts/no-evid...,false
24989,Eric Schmitt “sponsored a bill to spend $480 m...,Team PAC,/factchecks/2022/feb/09/team-pac/race-open-us-...,half-true


In [19]:
print(politifact_df.info())
politifact_df.tail(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24990 entries, 0 to 24989
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   statement  24990 non-null  object
 1   source     24990 non-null  object
 2   link       24990 non-null  object
 3   veracity   24990 non-null  object
dtypes: object(4)
memory usage: 781.1+ KB
None


,statement,source,link,veracity
24988,Comedian Heather McDonald collapsed on stage a...,Facebook posts,/factchecks/2022/feb/09/facebook-posts/no-evid...,false
24989,Eric Schmitt “sponsored a bill to spend $480 m...,Team PAC,/factchecks/2022/feb/09/team-pac/race-open-us-...,half-true


In [20]:
politifact_df.to_csv('politifact.csv')